# Processing

In this step we generate sparse matrix, general information and ICD9 codes for each patient.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re
import traceback
import shutil
from collections import OrderedDict

from utils import getConnection
from utils import parseUnitsMap
from utils import parseNum
from utils import sparsify

# %matplotlib inline

## Set unit conversion map

We manually set rules for unit conversion and store them in file './config/unitsmap.unit'. The file format is as following:

```
tablename:[name of table in database]
[itemid1],[target unit1],[unit11:ratio11],[unit12:ratio12],...,[unit1n:ratio1n]
...
[itemidn],[target unitn],[unitn1:ration1],[unitn2:ration2],...,[unitnn:rationn]
```

The ratio is set using the following rule: $\mathrm{unit1}*\mathrm{ratio1}=\mathrm{unit2}*\mathrm{ratio2}=...=\mathrm{unitn}*\mathrm{ration}$. For example, one row in the file could be: `227114,mg,mg:1,mcg:1000`. It means that $1\mathrm{mg}=1000\mathrm{mcg}$.

In [2]:
conn = getConnection()
UNITSMAP = parseUnitsMap()

In [3]:
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

_adm_first = np.load('res/admission_first_ids.npy').tolist()
admission_first_ids = _adm['admission_ids']

In [4]:
v = np.load('res/filtered_input.npy').tolist()
valid_input = v['id']
valid_input_unit = v['unit']

v = np.load('res/filtered_output.npy').tolist()
valid_output = v['id']

v = np.load('res/filtered_chart.npy').tolist()
valid_chart = v['id']
valid_chart_unit = v['unit']

v = np.load('res/filtered_chart_num.npy').tolist()
valid_chart_num = v['id']
valid_chart_num_unit = v['unit']

v = np.load('res/filtered_chart_cate.npy').tolist()
valid_chart_cate = v['id']

v = np.load('res/filtered_chart_ratio.npy').tolist()
valid_chart_ratio = v['id']

v = np.load('res/filtered_lab.npy').tolist()
valid_lab = v['id']
valid_lab_unit = v['unit']

v = np.load('res/filtered_lab_num.npy').tolist()
valid_lab_num = v['id']
valid_lab_num_unit = v['unit']

v = np.load('res/filtered_lab_cate.npy').tolist()
valid_lab_cate = v['id']

v = np.load('res/filtered_lab_ratio.npy').tolist()
valid_lab_ratio = v['id']

v = np.load('res/filtered_microbio.npy').tolist()
valid_microbio = v['id']

v = np.load('res/filtered_prescript.npy').tolist()
valid_prescript = v['id']
valid_prescript_unit = v['unit']

allids = valid_input+valid_output+valid_chart+valid_chart_num+valid_chart_cate+valid_chart_ratio+valid_chart_ratio+valid_lab+valid_lab_num+valid_lab_cate+valid_lab_ratio+valid_lab_ratio+valid_microbio+valid_prescript
print(len(allids), len(set(allids)))

23634 23533


## Create temporary tables for accelerating the query

In [ ]:
# put valid ids into database
conn = getConnection()
cur = conn.cursor()
for itemidlist, itemidlistname in zip([valid_input, valid_output, valid_chart, valid_chart_num, valid_chart_cate, valid_chart_ratio, valid_lab, valid_lab_num, valid_lab_cate, valid_lab_ratio], 'valid_input, valid_output, valid_chart, valid_chart_num, valid_chart_cate, valid_chart_ratio, valid_lab, valid_lab_num, valid_lab_cate, valid_lab_ratio'.replace(' ', '').split(',')):
    sql = 'drop table if exists mengcztemp_itemids_{0}'.format(itemidlistname)
    cur.execute(sql)
    conn.commit()
    sql = 'create table if not exists mengcztemp_itemids_{0} (\
    itemid serial PRIMARY KEY \
    )'.format(itemidlistname)
    cur.execute(sql)
    conn.commit()
    for itemid in itemidlist:
        sql = 'insert into mengcztemp_itemids_{0} (itemid) values ({1})'.format(itemidlistname, itemid)
        cur.execute(sql)
    conn.commit()
    sql = 'select * from mengcztemp_itemids_{0} limit 100'.format(itemidlistname)
    cur.execute(sql)
    res = cur.fetchall()
#     print(res)

In [ ]:
cur = conn.cursor()
sql = 'drop table if exists mengcztemp_itemids_{0}'.format('valid_prescript')
cur.execute(sql)
conn.commit()
sql = 'create table if not exists mengcztemp_itemids_{0} (\
    itemid varchar(255) PRIMARY KEY \
    )'.format('valid_prescript')
cur.execute(sql)
conn.commit()
for itemid in valid_prescript:
    sql = 'insert into mengcztemp_itemids_{0} (itemid) values (\'{1}\')'.format('valid_prescript', itemid)
    cur.execute(sql)
conn.commit()
sql = 'select * from mengcztemp_itemids_{0} limit 100'.format('valid_prescript')
cur.execute(sql)
res = cur.fetchall()
print(res, len(res), len(valid_prescript))

In [5]:
print('len(valid_input) = '+ str(len(valid_input)))
print('len(valid_output) = '+ str(len(valid_output)))
print ('len(valid_chart) = '+ str(len(valid_chart)))
print('len(valid_chart_num) = '+ str(len(valid_chart_num)))
print('len(valid_chart_cate) = '+ str(len(valid_chart_cate)))
print('len(valid_chart_ratio) = '+ str(len(valid_chart_ratio)))
print('len(valid_lab) = '+ str(len(valid_lab)))
print('len(valid_lab_num) = '+ str(len(valid_lab_num)))
print('len(valid_lab_cate) = '+ str(len(valid_lab_cate)))
print('len(valid_lab_ratio) = '+ str(len(valid_lab_ratio)))
print('len(valid_microbio) = '+ str(len(valid_microbio)))
print('len(valid_prescript) = '+ str(len(valid_prescript)))
print('\nlen(allids) = '+ str(len(allids)))

len(valid_input) = 3210
len(valid_output) = 1155
len(valid_chart) = 2644
len(valid_chart_num) = 314
len(valid_chart_cate) = 3405
len(valid_chart_ratio) = 100
len(valid_lab) = 372
len(valid_lab_num) = 113
len(valid_lab_cate) = 224
len(valid_lab_ratio) = 1
len(valid_microbio) = 9154
len(valid_prescript) = 2841

len(allids) = 23634


In [6]:
# map itemids to [0..n] column
index = 0
map_itemid_index = {}
allitem = allids
allitem_unit = valid_input_unit + ['NOCHECK'] * len(valid_output) + valid_chart_unit + valid_chart_num_unit + ['NOCHECK'] * len(valid_chart_cate) + ['NOCHECK'] * 2 * len(valid_chart_ratio) + valid_lab_unit + valid_lab_num_unit+ ['NOCHECK']*len(valid_lab_cate)+ ['NOCHECK'] * 2 * len(valid_lab_ratio)+['NOCHECK']*len(valid_microbio) + valid_prescript_unit
for i in range(len(allitem_unit)):
    allitem_unit[i] = allitem_unit[i].replace(' ','').lower()
assert len(allitem) == len(allitem_unit)
for ai in allitem:
    if ai not in map_itemid_index.keys():
        map_itemid_index[ai] = [index]
    else:
        map_itemid_index[ai].append(index)
    index+=1
# print(map_itemid_index)
print(len(map_itemid_index))
np.save('res/map_itemid_index.npy', map_itemid_index)

23533


## Map strings in categorical features to integers and store them to a file

In [ ]:
catedict = {}

for i in valid_chart_cate:
    cur = conn.cursor()
    cur.execute('SELECT distinct value FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in (select * from admission_ids)')
    distinctval = cur.fetchall()
    mapping = {};
    ct = 1;
    for d in distinctval:
        mapping[d[0]]= ct;
        ct+=1;
    catedict[i] = mapping
    print(i)
    
for i in valid_lab_cate:
    cur = conn.cursor()
    cur.execute('SELECT distinct value FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in (select * from admission_ids)')
    distinctval = cur.fetchall()
    mapping = {};
    ct = 1;
    for d in distinctval:
        mapping[d[0]]= ct;
        ct+=1;
    catedict[i] = mapping
    print(i)
    
np.save('res/catedict.npy', catedict)
print('saved!')

In [7]:
catedict = np.load('res/catedict.npy').tolist()
# print(catedict)

In [ ]:
def convert_units(unitmap, src_unit, dst_unit, src_value, f):
    try:
        src_ratio = unitmap['umap'][src_unit]
        dst_ratio = unitmap['umap'][dst_unit]
    except:
        print('converterror: ', unitmap, src_unit, dst_unit, src_value, file=f)
        return None
    if src_ratio == 0:
        return None
    else:
        return float(src_value) / src_ratio * dst_ratio

## Processing inputevents

1. Discard records without starttime.
2. Discard records without amount.
3. If the itemid matches manual rules, convert it according to the rule; else only keep the main unit and discard all other records not having the main unit.

In [8]:
def processing_inputevents(aid, admittime, conn, f):
    cur = conn.cursor()
    sql = '''select starttime, itemid, amount, amountuom from mimiciii.inputevents_mv where amount>0 and hadm_id={0} and itemid in (select * from mengcztemp_itemids_valid_input)
union all
select charttime as starttime, itemid, amount, amountuom from mimiciii.inputevents_cv where amount>0 and hadm_id={0} and itemid in (select * from mengcztemp_itemids_valid_input)'''.format(aid)
    cur.execute(sql)
    inputevents = cur.fetchall()
    inputevents_wholedata = []
    for ie in inputevents:
        starttime, itemid, amount, amountuom = ie[0], ie[1], ie[2], ie[3]
        # discard values with no starttime
        if starttime is None:
            print('no starttime: ', ie, file=f)
            continue
        # discard values with no amount
        if amount is None:
            print('no amount: ', ie, file=f)
            continue
        # convert units...
        amountuom = amountuom.replace(' ', '').lower()
        unitmap = UNITSMAP['inputevents']
        mainunit = allitem_unit[map_itemid_index[itemid][0]]
        if itemid in unitmap.keys():
            dst_value = convert_units(unitmap[itemid], amountuom, mainunit, amount, f)
        else:
            if amountuom == mainunit:
                dst_value = float(amount)
            else:
                dst_value = None
        if dst_value is None:
            print('not convertible: ', ie, file=f)
            continue
        inputevents_wholedata.append(['ie', (starttime - admittime).total_seconds(), [starttime, itemid, dst_value, mainunit]])
    return inputevents_wholedata

# processing_inputevents(184834, datetime.datetime.now(), getConnection(), sys.stdout)

## Processing outputevents

We only need to discard records without starttime or value.

In [9]:
def processing_outputevents(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value FROM mimiciii.outputevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_output)')
    outputevents = cur.fetchall()
    wholedata = []
    for oe in outputevents : 
        #check date
        if(oe[0] == None):
            print('no starttime', oe, file=f)
            continue
        #discard none value
        if(oe[2] == None):
            print('no value', oe, file=f)
            continue
        # no need to check unit all is mL
        oe = list(oe)
        oe.append('ml')
        wholedata.append(['oe',(oe[0]-admittime).total_seconds(),oe])
    return wholedata

# processing_outputevents(184834, datetime.datetime.now(), getConnection(), f=sys.stdout)

## Processing chartevents/labevnets

1. Discard records without starttime or value/valueuom.
2. Process 4 kinds of chartevents/labevents features separately.
    1. valid numerical features(numerical features not needing parsing): only need to convert units.
    2. categorical features: only need to map strings to integers.
    3. possible numerical features(values need parsing):
        1. parse values
        2. convert units
    4. ratio features: store two numbers in the ratio separately.

In [10]:
def processing_chartevents(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,valuenum,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_chart)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents:     
        #check date
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
            continue
    
        #discard none value and none valueuom
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
    
        #tuple to list
        ce = list(ce)

        # convert units...
        if ce[3] is None:
            ce[3] = ''
        itemid, valuenum, valueuom = ce[1], ce[2], ce[3]
        valueuom = valueuom.replace(' ', '').lower()
        unitmap = UNITSMAP['chartevents']
        mainunit = allitem_unit[map_itemid_index[itemid][0]]
        ce[3] = mainunit
        if itemid in unitmap.keys():
            dst_value = convert_units(unitmap[itemid], valueuom, mainunit, valuenum, f)
        else:
            if valueuom == mainunit or valueuom == '':
                dst_value = float(valuenum)
            else:
                dst_value = None
        ce[2] = dst_value
        
        #discard none value
        if(ce[2] == None):
            print('not convertible: ', ce, file=f)
            continue
        
        wholedata.append(['ce',(ce[0]-admittime).total_seconds(),list(ce)])
    return wholedata

def processing_chartevents_cate(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_chart_cate)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents: 
        #check date
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
            continue
            
        #discard none value
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
        
        #tuple to list
        ce = list(ce)
        
        #map to num
        ce[2] = catedict[ce[1]][ce[2]]
        if ce[2] is None:
            continue
        wholedata.append(['cecate',(ce[0]-admittime).total_seconds(),list(ce)])
    return wholedata
        
def processing_chartevents_num(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_chart_num)')
    chartevents = cur.fetchall()
    wholedata = []
    # parse numerical values
    for ce in chartevents:
        #tuple to list
        ce = list(ce)
        
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
        ce2res = parseNum(ce[2])
        
        #check date
        if(ce[0] == None):
            print('no starttime: ', ce, file=f)
            continue

        #discard none value
        if(ce2res == None):
#             writeline(f,'lenum None value :' + str(le))
            print('not parsed: ', ce, file=f)
            continue
        else:
            ce[2] = ce2res
    
        #check unit
        unitmap = UNITSMAP['chartevents']
        if ce[3] is None:
            ce[3] = ''
        currentunit = ce[3].replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        mainunit = allitem_unit[map_itemid_index[ce[1]][0]]
        if( currentunit == mainunit or currentunit == ''):
            pass
        else:    
            if ce[1] in unitmap.keys():
                ce[2] = convert_units(unitmap[ce[1]], currentunit, mainunit, ce[2], f)
            else:
                if currentunit != mainunit:
                    ce[2] = None
        
        #discard none value
        if(ce[2] == None):
            print('not convertible: ', ce, file=f)
            continue
        
        wholedata.append(['cenum',(ce[0]-admittime).total_seconds(),list(ce)])
    return wholedata

def processing_chartevents_ratio(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_chart_ratio)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents:
        ce = list(ce)
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
            continue
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
        try:
            fs = ce[2].split('/')
            f1, f2 = fs[0], fs[1]
            if f1 != '':
                ce[2] = float(f1)
                wholedata.append(['leratio_1', (ce[0] - admittime).total_seconds(), list(ce)])
            if f2 != '':
                ce[2] = float(f2)
                wholedata.append(['leratio_2', (ce[0] - admittime).total_seconds(), list(ce)])
        except:
            print('not parsed: ', ce, file=f)
            continue
    return wholedata
        
# processing_chartevents_ratio(136796, datetime.datetime.now(), getConnection(), f=sys.stdout)

In [11]:
def processing_labevents(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,valuenum,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_lab)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents:     
        #check date
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
            continue
    
        #discard none value
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
    
        #tuple to list
        ce = list(ce)

        # convert units...
        if ce[3] is None:
            ce[3] = ''
        itemid, valuenum, valueuom = ce[1], ce[2], ce[3]
        valueuom = valueuom.replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        unitmap = UNITSMAP['labevents']
        mainunit = allitem_unit[map_itemid_index[itemid][0]]
        ce[3] = mainunit
        if itemid in unitmap.keys():
            dst_value = convert_units(unitmap[itemid], valueuom, mainunit, valuenum, f)
        else:
            if valueuom == mainunit or valueuom == '':
                dst_value = float(valuenum)
            else:
                dst_value = None
        ce[2] = dst_value
        
        #discard none value
        if ce[2] is None:
            print('not convertible: ', ce, file=f)
            continue
        
        wholedata.append(['le',(ce[0]-admittime).total_seconds(),list(ce)])
    return wholedata

def processing_labevents_cate(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_lab_cate)')
    labevents = cur.fetchall()
    wholedata = []
    for le in labevents: 
        #check date
        if le[0] is None:
            print('no starttime: ', le, file=f)
            continue
            
        #discard none value
        if le[2] is None:
            print('no value: ', le, file=f)
            continue
        
        #tuple to list
        le = list(le)
        
        #map to num
        le[2] = catedict[le[1]][le[2]]
        if le[2] is None:
            continue
        wholedata.append(['lecate',(le[0]-admittime).total_seconds(),list(le)])
    return wholedata

def processing_labevents_num(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_lab_num)')
    labevents = cur.fetchall()
    wholedata = []
    # parse numerical values
    for le in labevents:
        #tuple to list
        le = list(le)
        
        if le[0] is None:
            print('no starttime', le, file=f)
            continue
        
        if le[2] is None:
            print('no value: ', le, file=f)
            continue
        
        #translate values
        le2res = parseNum(le[2])

        #discard none value
        if le2res is None:
            print('not parsed: ', le, file=f)
            continue
        else:
            le[2] = le2res
    
        #check unit
        unitmap = UNITSMAP['labevents']
        if le[3] is None:
            le[3] = ''
        currentunit = le[3].replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        mainunit = allitem_unit[map_itemid_index[le[1]][0]]
        if( currentunit == mainunit or currentunit == ''):
            pass
        else:    
            if le[1] in unitmap.keys():
                le[2] = convert_units(unitmap[le[1]], currentunit, mainunit, le[2], f)
            else:
                if currentunit != mainunit:
                    le[2] = None
        
        #discard none value
        if(le[2] == None):
            print('not convertible: ', le, file=f)
            continue
        
        wholedata.append(['lenum',(le[0]-admittime).total_seconds(),list(le)])
    return wholedata

def processing_labevents_ratio(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_lab_ratio)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents:
        ce = list(ce)
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
        try:
            fs = ce[2].split('/')
            f1, f2 = fs[0], fs[1]
            if f1 != '':
                ce[2] = float(f1)
                wholedata.append(['leratio_1', (ce[0] - admittime).total_seconds(), list(ce)])
            if f2 != '':
                ce[2] = float(f2)
                wholedata.append(['leratio_2', (ce[0] - admittime).total_seconds(), list(ce)])
        except:
            print('not parsed: ', ce, file=f)
            continue
    return wholedata

# processing_labevents_ratio(145834, datetime.datetime.now(), getConnection(), sys.stdout)

## Process microbiologyevents

1. Discard records without starttime.
2. Parse dose value in dilution_text. Values which can be parsed only contains '<'/'>'/'=' and numbers.

In [ ]:
def processing_microbiologyevents(aid, admittime, conn, f):
    wholedata = []
    for m in valid_microbio:
        cur = conn.cursor()
        sql = 'SELECT charttime,(spec_itemid,org_itemid,ab_itemid),dilution_text,\'uom\' FROM mimiciii.microbiologyevents WHERE hadm_id = '+str(aid)
        m = list(map(str, m))
        if(m[0] == 'None'):
            sql += ' and spec_itemid is null'
        else:
            sql += ' and spec_itemid = '+m[0]
    
        if(m[1] == 'None'):
            sql += ' and org_itemid is null'
        else:
            sql += ' and org_itemid = '+m[1]
        
        if(m[2] == 'None'):
            sql += ' and ab_itemid is null'
        else:
            sql += ' and ab_itemid = '+m[2]

        cur.execute(sql)
        microevents = cur.fetchall()
        for me in microevents: 
            me = list(me)
            for x in range(len(m)):
                try:
                    m[x] = int(m[x])
                except:
                    m[x] = None
            me[1] = tuple(m)
            #checkdate
            if(me[0] == None):
#                 writeline(f,'me date 0 : '+ " : "+str(me))
                print('no starttime: ', me, file=f)
                continue;

            #discard none value
            if(me[2] == None):
#                 writeline(f,'MICRO None value :' + str(me))
                print('no value: ', me, file=f)
                continue;
            
            #tuple to list
            me = list(me)
            
            #formatting 
            dose = me[2]
            dose = dose.replace('<','').replace('>','').replace('=','')
            numVal = None;
            if(dose == ''):
#                 writeline(f,'me parse fail : '+ " : "+str(me))
                print('not parsed: ', me, file=f)
                continue;
            try:
                numVal = float(dose)
            except:
#                 writeline(f,'me parse fail : '+ " : "+str(me))
                print('not parsed: ', me, file=f)
                continue;

            me[2]=numVal;
            
            #discard none value; check again after process
            if(me[2] == None):
#                 writeline(f,'MICRO None value :' + str(me))
                print('not parsed: ', me, file=f)
                continue;
                    
            wholedata.append(['me',(me[0]-admittime).total_seconds(),list(me)])
    return wholedata
            
# processing_microbiologyevents(149416, datetime.datetime.now(), getConnection(), sys.stdout)

## Process prescriptionevents

1. Discard records without starttime.
2. Parse values. Values containing only ','/'<'/'>'/'='/' ' and numbers can be parsed.
3. Discard all none values.
4. Convert units.

In [12]:
def processing_prescriptionevents(aid, admittime, conn, f):
    wholedata = []
    cur = conn.cursor()
    cur.execute('SELECT startdate,formulary_drug_cd,dose_val_rx,dose_unit_rx FROM mimiciii.prescriptions WHERE hadm_id = '+str(aid)+' and formulary_drug_cd in (select * from mengcztemp_itemids_valid_prescript)')
    presevents = cur.fetchall()
    for pe in presevents: 
        #checkdate
        if(pe[0] == None):
#             writeline(f,'pe date 0 : '  + " : "+str(pe))
            print('no starttime: ', pe, file=f)
            continue;
        
        #tuple to list
        pe = list(pe)
        
        if pe[2] is None:
            print('no value: ', pe, file=f)
            continue
        
        # formatting the value
        dose = pe[2]
        dose = dose.replace(',','').replace('<','').replace('>','').replace('=','').replace(' ','')
        numVal = None;
        try:
            numVal = float(dose)
        except:
            if(len(dose.split('-'))==2):
                strs = dose.split('-')
                try:
                    numVal = (float(strs[0]) + float(strs[1]))/2.0
                except:
#                     writeline(f,'pe parse fail : '  + " : "+str(pe))
                    print('not parsed: ', pe, file=f)
                    continue;
            else:
#                 writeline(f,'pe parse fail : '  + " : "+str(pe))
                print('not parsed: ', pe, file=f)
                continue;
        
        pe[2] = numVal;
        
        #discard none value
        if(pe[2] == None):
#             writeline(f,'PRES None value :' + str(pe))
            print('not parsed: ', pe, file=f)
            continue;
            
        #check unit
        # convert units...
        if pe[3] is None:
            pe[3] = ''
        itemid, valuenum, valueuom = pe[1], pe[2], pe[3]
        valueuom = valueuom.replace(' ','').lower()
        mainunit = allitem_unit[map_itemid_index[itemid][0]]
        if valueuom == mainunit or valueuom == '':
            dst_value = float(valuenum)
        else:
            dst_value = None
        pe[2] = dst_value

        #discard none value
        if(pe[2] == None):
#             writeline(f, 'PRES None value :' + str(pe))
            print('not convertible: ', pe, file=f)
            continue;
        
        wholedata.append(['pe',(pe[0]-admittime).total_seconds(),list(pe)])
    return wholedata

processing_prescriptionevents(185777, datetime.datetime.now(), getConnection(), sys.stdout)

[['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'GLAR100I', 6.0, 'UNIT']],
 ['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'BENZ100', 100.0, 'mg']],
 ['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'INSULIN', 0.0, 'UNIT']],
 ['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'GLAR100I', 6.0, 'UNIT']],
 ['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'PRIM263', 52.6, 'mg']],
 ['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'VANC1F', 1000.0, 'mg']],
 ['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'MAGS1I', 1.0, 'gm']],
 ['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'MICROK10', 60.0, 'mEq']],
 ['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'INSULIN', 0.0, 'UNIT']],
 ['pe',
  5480806544.761877,
  [datetime.datetime(2191, 3, 16, 0, 0), 'CLON5', 0.5, 'mg']],
 ['pe',
  5480806544.761877,
  [datetime.date

## Processing

In this step, we generate the matrix of time series for each admission.

1. Discard admissoins without admittime.
2. Collect records from inputevents, outputevents, chartevents, labevents, microbiologyevents and prescriptionevents.
3. For possible conflictions(different values of the same feature occur at the same time):
    1. For numerical values:
        1. For inputevents/outputevents/prescriptions, we use the sum of all conflicting values.
        2. For labevents/chartevents/microbiologyevents, we use the mean of all conflicting values.
    2. For categorical values: we use the value appear first and record that confliction event in the log.
    3. For ratio values: we separate the ratio to two numbers and use the mean of each of them.

In [13]:
# integrate the time series array for a single patient
def processing(aid, f):
    conn = getConnection()
    
    # get admittime
    cur = conn.cursor()
    cur.execute('select admittime from mimiciii.admissions where hadm_id={0}'.format(aid))
    admission = cur.fetchone()
    if admission is None:
        return None
    admittime = admission[0]
    if admittime is None:
        return None
    wholedata = []
    
    # preprocess inputevents
    wholedata.append(processing_inputevents(aid, admittime, conn, f))
    
    # preprocess outputevents
    wholedata.append(processing_outputevents(aid, admittime, conn, f))
    
    # preprocess chartevents
    wholedata.append(processing_chartevents(aid, admittime, conn, f))
    wholedata.append(processing_chartevents_cate(aid, admittime, conn, f))
    wholedata.append(processing_chartevents_num(aid, admittime, conn, f))
    wholedata.append(processing_chartevents_ratio(aid, admittime, conn, f))
    
    # preprocess labevents
    wholedata.append(processing_labevents(aid, admittime, conn, f))
    wholedata.append(processing_labevents_cate(aid, admittime, conn, f))
    wholedata.append(processing_labevents_num(aid, admittime, conn, f))
    wholedata.append(processing_labevents_ratio(aid, admittime, conn, f))
    
    # preprocess microbiologyevents
    wholedata.append(processing_microbiologyevents(aid, admittime, conn, f))
    
    # preprocess prescriptionevents
    wholedata.append(processing_prescriptionevents(aid, admittime, conn, f))
    
    # here is the sparse matrix, order by timestamp
    wholedata = sorted(list(itertools.chain(*wholedata)), key=itemgetter(1))
    
    # transform sparse matrix to matrix
    D = len(allids) + 2
    
    # map time to row
    map_time_index = {}
    index = 0
    for wd in wholedata:
        if(wd[1] not in map_time_index):
            map_time_index[wd[1]] = index
            index += 1
            
    patient = [[None for i in range(D)] for j in range(len(map_time_index))]
    numtodivide = [[0 for i in range(D-2)] for j in range(len(map_time_index))]
#     writeline(f,'len(wholedata) = '+str(len(wholedata)))
#     writeline(f, 'D = '+str(D))
#     writeline(f,'len(patient) = '+str(len(patient)) +' timesteps')

    for wd in wholedata:

        assert patient[ map_time_index[wd[1]] ][D-2] == None or patient[ map_time_index[wd[1]] ][D-2] == wd[1]
        patient[ map_time_index[wd[1]] ][D-2] = wd[1]
        patient[ map_time_index[wd[1]] ][D-1] = aid

        if(wd[0] ==  'ie' or wd[0] ==  'oe' or wd[0] ==  'pe'):
            if(patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][0]] ==None):
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][0]] = wd[2][2] 
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][0]] += wd[2][2]

        if(wd[0] ==  'le' or wd[0] == 'ce' or wd[0] == 'me' or wd[0] ==  'lenum' or wd[0] == 'cenum'):
            if wd[2][2] is None:
                print('None value: ', wd, file=f)
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]][0]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]][0]  ] = wd[2][2]
                numtodivide[map_time_index[wd[1]]  ]  [  map_itemid_index[wd[2][1]][0] ] = 1
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][0]] += wd[2][2]
                numtodivide[ map_time_index[wd[1]]  ] [   map_itemid_index[wd[2][1]][0] ] += 1
                
        if (wd[0].startswith('ceratio') or wd[0].startswith('leratio')):
            ot = int(wd[0].split('_')[1]) - 1
            if wd[2][2] is None:
                print(wd, file=f)
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]][ot]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]][ot]  ] = wd[2][2]
                numtodivide[map_time_index[wd[1]]  ]  [  map_itemid_index[wd[2][1]][ot] ] = 1
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][ot]] += wd[2][2]
                numtodivide[ map_time_index[wd[1]]  ] [   map_itemid_index[wd[2][1]][ot] ] += 1
        
        if(wd[0] == 'cecate' or wd[0] == 'lecate'):
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]][0]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]][0]  ] = wd[2][2]
            else: 
                print('DUPLICATED :',wd, file=f)
        
    for i in range(len(map_time_index)):
        for j in range(D-2):
            if(numtodivide[i][j] == 0): continue;
            try:
                patient[i][j] /= numtodivide[i][j]
            except:
                print('div error: ', i, j, file=f)
    conn.close()
    return patient

## Generate information of patient

Here we collect information of one patient, containing its admission_type, admission_location, insurance, language, religion, marital_status and ethnicity.

Since all of them are categorical features, we map the strings of each feature to integers and store the mapping.

In [14]:
# generate general information of patient

# generate map for categorical values
conn = getConnection()
cate = ['admission_type','admission_location','insurance','language','religion','marital_status','ethnicity']
mapping = {}
for c in cate:
    cur = conn.cursor()
    cur.execute('select distinct '+c+' from mimiciii.admissions')
    types = cur.fetchall()
    
    catemapping = {}
    for i in range(len(types)):
        catemapping[types[i][0]] = i 
    mapping[c] = catemapping

# add map for services
cur = conn.cursor()
cur.execute('select distinct ' + 'curr_service' + ' from mimiciii.services')
types = cur.fetchall()

catemapping = {}
for i, typen in enumerate(types):
    catemapping[typen[0]] = i
mapping['curr_service'] = catemapping
# mapping
mapping['curr_service']
np.save('res/adm_catemappings.npy', mapping)
mapping

{'admission_location': {'** INFO NOT AVAILABLE **': 4,
  'CLINIC REFERRAL/PREMATURE': 3,
  'EMERGENCY ROOM ADMIT': 0,
  'HMO REFERRAL/SICK': 7,
  'PHYS REFERRAL/NORMAL DELI': 8,
  'TRANSFER FROM HOSP/EXTRAM': 1,
  'TRANSFER FROM OTHER HEALT': 2,
  'TRANSFER FROM SKILLED NUR': 5,
  'TRSF WITHIN THIS FACILITY': 6},
 'admission_type': {'ELECTIVE': 0, 'EMERGENCY': 2, 'NEWBORN': 3, 'URGENT': 1},
 'curr_service': {'CMED': 5,
  'CSURG': 17,
  'DENT': 6,
  'ENT': 10,
  'GU': 4,
  'GYN': 8,
  'MED': 3,
  'NB': 7,
  'NBB': 14,
  'NMED': 16,
  'NSURG': 0,
  'OBS': 2,
  'OMED': 11,
  'ORTHO': 13,
  'PSURG': 12,
  'PSYCH': 19,
  'SURG': 15,
  'TRAUM': 9,
  'TSURG': 1,
  'VSURG': 18},
 'ethnicity': {'AMERICAN INDIAN/ALASKA NATIVE': 35,
  'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE': 14,
  'ASIAN': 12,
  'ASIAN - ASIAN INDIAN': 21,
  'ASIAN - CAMBODIAN': 32,
  'ASIAN - CHINESE': 26,
  'ASIAN - FILIPINO': 19,
  'ASIAN - JAPANESE': 11,
  'ASIAN - KOREAN': 16,
  'ASIAN - OTHER': 23,
  'ASI

## Generate non-temporal features

Here we collect all non-temporal features only related to the admissions:
1. admission id
2. subject id(for finding the patient of one admission)
3. age(at admittime, unit is day)
4. length of stay(unit is minute)
5. in-hospital mortality label
6. labelGurantee label
7. 1-day mortality(from admittime)
8. 2-day mortality(from admittime)
9. 3-day mortality(from admittime)
10. 30-day mortality(from dischtime)
11. 1-year mortality(from dischtime)
12. admission_type
13. admission_location
14. insurance
15. language
16. religion
17. marital_status
18. ethnicity

**Mortality label here is not used, please refer to 8_collect_time_labels.ipynb to get correct mortality labels. We leave them here only for compatibility.**

In [32]:
def ageLosMortality(aid, f):
    conn = getConnection()
    
    cur = conn.cursor()
    cur.execute('SELECT hadm_id,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,insurance,language,religion,marital_status,ethnicity FROM mimiciii.ADMISSIONS WHERE hadm_id='+str(aid))
    admission = cur.fetchone()
    
    assert admission!=None
    
    subject_id = admission[1]
    admittime = admission[2]
    dischtime = admission[3]
    deathtime = admission[4]
    
    cur = conn.cursor()
    cur.execute('SELECT dob, dod FROM mimiciii.PATIENTS WHERE subject_id='+str(subject_id))
    patient = cur.fetchone()

    assert patient!=None
    birthdate = patient[0]
    final_deathdate = patient[1]
    mortal = 0
    labelGuarantee = 0
    die24 = 0
    die24_48 = 0
    die48_72 = 0
    die30days = 0
    die1year = 0
    if(deathtime != None) : 
        mortal = 1
        if(deathtime != dischtime):
            labelGuarantee=1
        secnum = (deathtime - admittime).total_seconds()
        if secnum <= 24 * 60 * 60:
            die24 = 1
        if secnum <= 48 * 60 * 60:
            die24_48 = 1
        if secnum <= 72 * 60 * 60:
            die48_72 = 1
    if dischtime is not None and final_deathdate is not None:
        dischsecnum = (final_deathdate - dischtime).total_seconds()
        if dischsecnum <= 30 * 24 * 60 * 60:
            die30days = 1
        if dischsecnum <= 365 * 24 * 60 * 60:
            die1year = 1
            
    cur.execute('select curr_service from mimiciii.services where hadm_id='+str(aid))
    curr_service = cur.fetchone()
    if curr_service:
        curr_service = curr_service[0]
    else:
        curr_service = 'NB'

    data = [aid,subject_id, (admittime - birthdate).total_seconds()/(3600*24), (dischtime-admittime).total_seconds()//60., mortal, labelGuarantee, die24, die24_48, die48_72, die30days, die1year, mapping['curr_service'][curr_service]]
    for i in range(5,12):
        data.append(mapping[cate[i-5]][admission[i]])
    conn.close()
    return data

# ageLosMortality(128652, sys.stdout)

## Generate ICD9 codes

Here we convert icd9 codes to category numbers.

In [16]:
def ICD9(aid, f):
    conn = getConnection()
    cate20=0
    
    cur = conn.cursor()
    cur.execute('SELECT icd9_code FROM mimiciii.DIAGNOSES_ICD WHERE hadm_id='+str(aid)+' ORDER BY seq_num')
    icd9s = cur.fetchall() 
    list_icd9 = []
    for icd9 in icd9s:
        icd = icd9[0];
        if icd is None:
            continue
        if(icd[0] =='V'):
            label_name = 19
            numstr = icd[0:3]+'.'+icd[3:len(icd)]
        elif(icd[0] =='E'):
            cate20 +=1
            label_name = 20
            numstr = icd
        else:
            num = float(icd[0:3])
            numstr = icd[0:3]+'.'+icd[3:len(icd)]
            if(num >= 1 and num <= 139 ) :
                label_name = 0
            if(num >= 140 and num <= 239 ) :
                label_name = 1
            if(num >= 240 and num <= 279 ) :
                label_name = 2
            if(num >= 280 and num <= 289 ) :
                label_name = 3
            if(num >= 290 and num <= 319 ) :
                label_name = 4
            if(num >= 320 and num <= 389 ) :
                label_name = 5
            if(num >= 390 and num <= 459 ) :
                label_name = 6
            if(num >= 460 and num <= 519 ) :
                label_name = 7
            if(num >= 520 and num <= 579 ) :
                label_name = 8
            if(num >= 580 and num <= 629 ) :
                label_name = 9
            if(num >= 630 and num <= 677 ) :
                label_name = 10
            if(num >= 680 and num <= 709 ) :
                label_name = 11
            if(num >= 710 and num <= 739 ) :
                label_name = 12
            if(num >= 740 and num <= 759 ) :
                label_name = 13
            if(num >= 760 and num <= 779 ) :
                label_name = 14
            if(num >= 780 and num <= 789 ) :
                label_name = 15
            if(num >= 790 and num <= 796 ) :
                label_name = 16
            if(num >= 797 and num <= 799 ) :
                label_name = 17
            if(num >= 800 and num <= 999 ) :
                label_name = 18
        list_icd9.append([aid,icd,numstr,label_name])
    conn.close()
    return list_icd9

# ICD9(185777, sys.stdout)

In [ ]:
if os.path.exists('admdata/'):
    shutil.rmtree('admdata/')
os.makedirs('admdata/log/')

## Save one file for each admission

For each admission, we save a separate file for it, which contains:
1. 'timeseries': matrix of time series in form of sparse matrix
2. 'general': non-temporal features
3. 'icd9': list of icd9 category codes

In [17]:
def process_patient(aid):
    with open('admdata/log/adm-{0}.log'.format(str('%.6d' % aid)), 'w') as f:
        try:
            proc = processing(aid, f)
            if len(proc) == 0:
                return
            res = {
                'timeseries': sparsify(proc),
                'general': ageLosMortality(aid, f),
                'icd9': ICD9(aid, f)
            }
            np.save('admdata/adm-' + str('%.6d' % aid), res)
            print('finished {0}!'.format(aid))
        except Exception as e:
            with open('admdata/log/admerror-{0}.log'.format(str('%.6d' % aid)), 'w') as ferr:
                traceback.print_exc(file=ferr)
            traceback.print_exc(sys.stdout)
            print('failed at {0}!'.format(aid))
    
# process_patient(136796)

In [33]:
# add labels about mortality, now we have 1day|2days|3days|in-hospitial|30days|1year
def add_mortality_labels(aid):
    with open('admdata/log/adm-addml{0}.log'.format(str('%.6d' % aid)), 'w') as f:
        try:
            res = np.load('admdata/adm-' + str('%.6d' % aid)+'.npy').tolist()
        except Exception as e:
            traceback.print_exc(file=f)
            return
        res['general'] = ageLosMortality(aid, f)
        np.save('admdata/adm-' + str('%.6d' % aid)+'.npy', res)
    print('finished {0}!'.format(aid))

p = Pool()
for aid in admission_ids:
    p.apply_async(add_mortality_labels, args=(aid,))
p.close()
p.join()
# t = 0
# for aid in admission_ids:
#     add_mortality_labels(aid, sys.stdout)
#     t += 1
#     if t % 100 == 0:
#         print(t)

finished 171091!
finished 119129!
finished 191210!
finished 113210!
finished 145127!
finished 115178!
finished 164444!
finished 186720!
finished 108031!
finished 191259!
finished 137515!
finished 128584!
finished 157477!
finished 157392!
finished 190267!
finished 174117!
finished 157075!
finished 170590!
finished 170691!
finished 128124!
finished 120113!
finished 104677!
finished 131243!
finished 175538!
finished 162412!
finished 107019!
finished 155964!
finished 178138!
finished 171412!
finished 193878!
finished 139826!
finished 199118!
finished 186894!
finished 112235!
finished 169842!
finished 119341!
finished 107667!
finished 173466!
finished 151200!
finished 179343!
finished 198726!
finished 136876!
finished 149342!
finished 103674!
finished 120234!
finished 176432!
finished 147586!
finished 100050!
finished 119375!
finished 124764!
finished 100347!
finished 139052!
finished 101095!
finished 115231!
finished 161569!
finished 138410!
finished 174729!
finished 162808!
finished 14878

finished 159908!
finished 131089!
finished 141577!
finished 123865!
finished 190245!
finished 134955!
finished 128677!
finished 154253!
finished 146483!
finished 165205!
finished 131753!
finished 141749!
finished 110667!
finished 191601!
finished 100439!
finished 187952!
finished 188490!
finished 194269!
finished 171663!
finished 116494!
finished 128970!
finished 185349!
finished 124399!
finished 142693!
finished 153271!
finished 157174!
finished 198727!
finished 105090!
finished 100325!
finished 124653!
finished 184592!
finished 176579!
finished 119930!
finished 106560!
finished 170416!
finished 181829!
finished 188872!
finished 171989!
finished 100769!
finished 186927!
finished 134307!
finished 129578!
finished 115493!
finished 188950!
finished 146538!
finished 120508!
finished 189333!
finished 115983!
finished 126437!
finished 130122!
finished 121893!
finished 185756!
finished 165488!
finished 181546!
finished 158419!
finished 101993!
finished 103194!
finished 127584!
finished 19677

finished 167747!
finished 103196!
finished 118249!
finished 162676!
finished 180714!
finished 148143!
finished 115910!
finished 105986!
finished 122851!
finished 164990!
finished 178496!
finished 168070!
finished 163259!
finished 130084!
finished 165859!
finished 144037!
finished 105944!
finished 159469!
finished 176849!
finished 166207!
finished 189669!
finished 183551!
finished 120237!
finished 145240!
finished 166136!
finished 111792!
finished 174063!
finished 134224!
finished 166643!
finished 127919!
finished 108915!
finished 169154!
finished 187014!
finished 186215!
finished 143040!
finished 103175!
finished 199767!
finished 143984!
finished 117382!
finished 173779!
finished 184523!
finished 150091!
finished 155792!
finished 136624!
finished 128694!
finished 174416!
finished 187675!
finished 163618!
finished 177171!
finished 108433!
finished 195037!
finished 118988!
finished 186069!
finished 176466!
finished 147884!
finished 159232!
finished 135227!
finished 129378!
finished 13963

finished 156525!
finished 132473!
finished 193165!
finished 148373!
finished 197069!
finished 131142!
finished 184545!
finished 161654!
finished 154415!
finished 176423!
finished 177919!
finished 128498!
finished 184138!
finished 176910!
finished 164403!
finished 127580!
finished 132107!
finished 162395!
finished 177498!
finished 134073!
finished 197504!
finished 179500!
finished 189989!
finished 188844!
finished 198129!
finished 162453!
finished 164162!
finished 109938!
finished 159884!
finished 115326!
finished 113111!
finished 190829!
finished 142586!
finished 137490!
finished 131592!
finished 165131!
finished 150883!
finished 163003!
finished 198156!
finished 114818!
finished 133223!
finished 146325!
finished 144816!
finished 154043!
finished 176757!
finished 125449!
finished 101157!
finished 174023!
finished 168478!
finished 180230!
finished 185724!
finished 112588!
finished 175782!
finished 171464!
finished 114089!
finished 162619!
finished 177083!
finished 191198!
finished 11461

finished 173270!
finished 122900!
finished 131438!
finished 177644!
finished 150592!
finished 116100!
finished 160150!
finished 138686!
finished 104675!
finished 150662!
finished 188372!
finished 159446!
finished 193461!
finished 179208!
finished 129591!
finished 146776!
finished 107848!
finished 168839!
finished 106697!
finished 197849!
finished 100836!
finished 143682!
finished 102206!
finished 142535!
finished 114472!
finished 119438!
finished 108624!
finished 106829!
finished 190428!
finished 198710!
finished 174984!
finished 176417!
finished 125731!
finished 193788!
finished 140491!
finished 162086!
finished 139820!
finished 132379!
finished 141202!
finished 189816!
finished 175701!
finished 182983!
finished 186041!
finished 166085!
finished 189387!
finished 150497!
finished 155170!
finished 178575!
finished 188981!
finished 192745!
finished 187738!
finished 166145!
finished 184917!
finished 164333!
finished 167826!
finished 197951!
finished 123000!
finished 160063!
finished 16367

...

finished 189910!
finished 194963!
finished 153932!
finished 158914!
finished 106375!
finished 170229!
finished 151740!
finished 174542!
finished 147856!
finished 192946!
finished 177337!
finished 103682!
finished 154094!
finished 157162!
finished 183744!
finished 174692!
finished 121582!
finished 150129!
finished 182307!
finished 126083!
finished 112153!
finished 168466!
finished 157196!
finished 166939!
finished 114175!
finished 155235!
finished 108846!
finished 179898!
finished 166960!
finished 145947!
finished 183074!
finished 113404!
finished 182562!
finished 145250!
finished 115713!
finished 153381!
finished 179653!
finished 156266!
finished 151544!
finished 111608!
finished 114824!
finished 147621!
finished 172317!
finished 160198!
finished 131658!
finished 184014!
finished 121829!
finished 161104!
finished 143712!
finished 143616!
finished 180179!
finished 197517!
finished 141845!
finished 136809!
finished 187931!
finished 156657!
finished 159913!
finished 107218!
finished 14124

finished 105411!
finished 195845!
finished 155140!
finished 142940!
finished 199445!
finished 151469!
finished 141021!
finished 192431!
finished 101148!
finished 124229!
finished 199105!
finished 146645!
finished 188530!
finished 198732!
finished 175848!
finished 155046!
finished 107337!
finished 143664!
finished 108068!
finished 171311!
finished 139360!
finished 162222!
finished 107093!
finished 103295!
finished 139478!
finished 160986!
finished 114962!
finished 131316!
finished 155281!
finished 193814!
finished 121321!
finished 181242!
finished 120556!
finished 135203!
finished 193971!
finished 152604!
finished 162732!
finished 120623!
finished 126407!
finished 190941!
finished 175428!
finished 185764!
finished 157056!
finished 172383!
finished 131193!
finished 108420!
finished 184997!
finished 196755!
finished 108441!
finished 128281!
finished 122002!
finished 179408!
finished 171604!
finished 105143!
finished 179985!
finished 148414!
finished 143408!
finished 150590!
finished 17160

finished 161915!
finished 188695!
finished 133047!
finished 184995!
finished 178216!
finished 122792!
finished 126812!
finished 107345!
finished 173323!
finished 170715!
finished 187154!
finished 115145!
finished 128174!
finished 137641!
finished 142337!
finished 122663!
finished 112635!
finished 125755!
finished 121921!
finished 104543!
finished 116721!
finished 174522!
finished 136883!
finished 149982!
finished 161757!
finished 143312!
finished 188560!
finished 122737!
finished 177617!
finished 154695!
finished 169392!
finished 167513!
finished 110846!
finished 145524!
finished 105027!
finished 150206!
finished 146302!
finished 168662!
finished 154487!
finished 165735!
finished 190900!
finished 147872!
finished 163110!
finished 193754!
finished 146244!
finished 148575!
finished 135943!
finished 119407!
finished 133797!
finished 173252!
finished 149010!
finished 104381!
finished 141979!
finished 148671!
finished 168974!
finished 110016!
finished 149207!
finished 158299!
finished 17715

finished 103254!
finished 194415!
finished 147295!
finished 190897!
finished 144697!
finished 113878!
finished 173544!
finished 115922!
finished 180176!
finished 194298!
finished 195307!
finished 141422!
finished 165111!
finished 117282!
finished 155003!
finished 112025!
finished 149039!
finished 192513!
finished 161279!
finished 191739!
finished 192084!
finished 164305!
finished 188180!
finished 169328!
finished 150214!
finished 120648!
finished 152752!
finished 170249!
finished 135882!
finished 169084!
finished 141037!
finished 134082!
finished 170564!
finished 186718!
finished 162933!
finished 151732!
finished 159338!
finished 153367!
finished 138100!
finished 124301!
finished 137594!
finished 175039!
finished 196318!
finished 141052!
finished 105645!
finished 123854!
finished 145174!
finished 185875!
finished 176553!
finished 166586!
finished 143648!
finished 156474!
finished 157404!
finished 122690!
finished 148412!
finished 187451!
finished 114063!
finished 178369!
finished 18613

finished 118667!
finished 132775!
finished 145235!
finished 112711!
finished 128383!
finished 175569!
finished 182243!
finished 102459!
finished 199464!
finished 146036!
finished 149438!
finished 169754!
finished 130595!
finished 144754!
finished 153625!
finished 172785!
finished 141862!
finished 121623!
finished 123128!
finished 188863!
finished 192962!
finished 189150!
finished 142132!
finished 105240!
finished 166883!
finished 172515!
finished 194856!
finished 116645!
finished 167967!
finished 128683!
finished 190625!
finished 133319!
finished 199880!
finished 149648!
finished 112008!
finished 145534!
finished 143829!
finished 134396!
finished 124991!
finished 154813!
finished 191729!
finished 173758!
finished 128516!
finished 194390!
finished 106243!
finished 183420!
finished 196643!
finished 191328!
finished 103518!
finished 152493!
finished 199533!
finished 170988!
finished 130451!
finished 150463!
finished 199018!
finished 148009!
finished 137588!
finished 124720!
finished 19922

In [ ]:
def copy_first_admissions(aid):
    if aid in admission_first_ids_set:
        res = np.load(os.path.join('admdata', 'adm-%.6d.npy' % aid))
        np.save(os.path.join('admdata_first', 'adm-%.6d.npy' % aid), res)
        
p = Pool()
p.map(copy_first_admissions, admission_ids)
p.close()
p.join()